# Aberrated Image Recovery of Ultracold Atoms

<img src="https://raw.githubusercontent.com/illinois-ipaml/MachineLearningForPhysics/main/img/Project_UltraColdAtoms.png" width=600 align=left></img><br>

## <span style="color:Orange">Overview</span>
Over the past 20-30 years the groundwork has been laid for precise experimental control of atomic gases at ultracold temperatures. These ultracold atom gas experiments explore the quantum mechanics of their underlying atomic systems to a diverse set applications ranging from the simulation of computationally difficult physics problems [1] to the sensing of new physics beyond the standard model [2]. Most experiments in this field rely on directly imaging the ultracold gas they make, in order to extract data about the size and shape of the atomic number density distribution being imaged.

The goal of this project is to introduce you to some relevant image processing techniques, as well as to get familiar with an image as a data element. As you will demonstrate, images are a kind of data with a very large number of features, but where almost all of those features within some region of interest are highly correlated. Of interest in this project is how the effects of real imaging systems distort the information contained in an image, and how those effects can be unfolded from the data to recover information about the true density distribution. 

Capturing all possible kinds of aberrations and noise present in real experimental data of ultracold atom images is outside the scope of the simulated test data in this project. Instead, we limit ourselves to a few key effects: optical aberrations in the form of defocus and primary spherical aberrations, pixelization from a finite detector resolution, and at times toy number density noise simulated as simple Gaussian noise.

---

## <span style="color:Orange">References</span>

* [__<span style="color:Red">1</span>__] S. S. Kondov, W. R. McGehee, J. J. Zirbel, and B. DeMarco, Science 334, 66 (2011). 

* [__<span style="color:Red">2</span>__] W. B. Cairncross, D. N. Gresh, M. Grau, K. C. Cossel, T. S. Roussy, Y. Ni, Y. Zhou, J. Ye, and E. A. Cornell, Physical Review Letters 119, (2017). 

* [__<span style="color:Red">3</span>__] C. J. Schuler, Machine learning approaches to image deconvolution. Eberhard Karls University of Tübingen (2014).

* [__<span style="color:Red">4</span>__] Herbel, J., Kacprzak, T., Amara, A., Refregier, A., & Lucchi, A. (2018). Fast point spread function modeling with deep learning. Journal of Cosmology and Astroparticle Physics, 2018(07), 054.

* [__<span style="color:Red">5</span>__] R. J. Steriti and M. A. Fiddy, "Blind deconvolution of images by use of neural networks," Opt. Lett. 19, 575-577 (1994)

---

## <span style="color:Orange">Acknowledgements</span>

* Initial version: Max Gold with some guidence from Mark Neubauer

© Copyright 2024